In [148]:
import json
from helper_synthesis import xml_to_json, check_valid_aspect_xml, count_sentences_in_text, has_empty_tag, has_aspect_term_of_invalid_pos_tags, remove_xml_tags_from_string

idx = 4
path_examples = f'../07 train models/synth/GPT-3/random/split_{idx}.json'
path_examples_out = f'../07 train models/synth/GPT-3/random/split_{idx}.json'

with open(path_examples, 'r', encoding="utf-8") as file:
    data_dictionary = json.load(file)

In [149]:
change_dictionary = data_dictionary.copy()

## Optimize Past Statistic

In [150]:
for i in range(0, len(data_dictionary)):
    invalid_xml_tags = 0
    aspect_polarity_in_text_but_not_in_label = 0
    no_token_in_sentence = 0
    more_than_one_sentences = 0
    empty_aspect_term = 0
    invalid_single_word_aspect_term_pos_tag = 0

    change = 0
    for k in range(0, len(data_dictionary[i]["llm_retry_statistic"])):
        if data_dictionary[i]["llm_retry_statistic"][k]["llm_invalid_xml_schema"] > change:
            change += 1
        else:

            prediction = data_dictionary[i]["llm_retry_statistic"][k]["llm_prediction_raw"]
            label = [tuple(l) for l in data_dictionary[i]
                     ["llm_retry_statistic"][k]["llm_label"]]

            # 00 check if valid aspect term names / attributes
            invalid_xml_names = False
            if check_valid_aspect_xml(f'<input>{prediction}</input>') == False:
                invalid_xml_names = True
                invalid_xml_tags += 1

            if invalid_xml_names:
                prediction_as_json = xml_to_json(
                    remove_xml_tags_from_string(prediction), label, "", 0)
            else:
                prediction_as_json = xml_to_json(prediction, label, "", 0)

            # 01 check if aspects are present in label
            if prediction_as_json == "not-in-label":
                prediction_as_json = xml_to_json(
                    prediction, label, "", 0, check_label=False)
                aspect_polarity_in_text_but_not_in_label += 1

            # 02 check if text is empty
            if len(prediction_as_json["text"]) == 0:
                valid_language_check = False
                no_token_in_sentence += 1

            # 03 count number of sentences
            if count_sentences_in_text(prediction_as_json["text"]) > 1:
                more_than_one_sentences += 1

            # 04 check if empty aspect term exists
            if has_empty_tag(prediction_as_json):
                empty_aspect_term += 1

            # 05 check for single word of type ADJ, ADV, AUX, CONJ, CCONJ, DET, INTJ, PART, PRON, SCONJ, VERB
            if has_aspect_term_of_invalid_pos_tags(prediction_as_json):
                invalid_single_word_aspect_term_pos_tag += 1

        change_dictionary[i]["llm_retry_statistic"][k]["llm_invalid_xml_tags"] = invalid_xml_tags
        change_dictionary[i]["llm_retry_statistic"][k]["llm_aspect_polarity_in_text_but_not_in_label"] = aspect_polarity_in_text_but_not_in_label
        change_dictionary[i]["llm_retry_statistic"][k]["llm_no_token_in_sentence"] = no_token_in_sentence
        change_dictionary[i]["llm_retry_statistic"][k]["llm_more_than_one_sentences"] = more_than_one_sentences
        change_dictionary[i]["llm_retry_statistic"][k]["llm_empty_aspect_term"] = empty_aspect_term
        change_dictionary[i]["llm_retry_statistic"][k]["llm_invalid_single_word_aspect_term_pos_tag"] = invalid_single_word_aspect_term_pos_tag

    change_dictionary[i]["llm_invalid_xml_tags"] = invalid_xml_tags
    change_dictionary[i]["llm_aspect_polarity_in_text_but_not_in_label"] = aspect_polarity_in_text_but_not_in_label
    change_dictionary[i]["llm_no_token_in_sentence"] = no_token_in_sentence
    change_dictionary[i]["llm_more_than_one_sentences"] = more_than_one_sentences
    change_dictionary[i]["llm_empty_aspect_term"] = empty_aspect_term
    change_dictionary[i]["llm_invalid_single_word_aspect_term_pos_tag"] = invalid_single_word_aspect_term_pos_tag

## Check Final Prediction

Nun schaue ich noch, ob bei der aktuellen validen prediction 

In [151]:
for i in range(len(data_dictionary)):
    if len(data_dictionary[i]["text"]) == 0:
        print(i)
        change_dictionary[i]["retry_no_token"] = True

In [152]:
with open(path_examples_out, "w", encoding="utf-8") as outfile:
    json.dump(change_dictionary, outfile, ensure_ascii=False)